In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sys import argv
from collections import defaultdict
from time import sleep
import os
import boto3
from io import StringIO, BytesIO
import re
from time import sleep
from urlparse import urljoin



In [37]:
## Look at the Data

# print soup.prettify() 

# print soup.title

# print soup.table.find_all('td')

# print soup.td.string

# jumble.find_all('td.string')

## print soup.find_all('td')


# rows = soup.select('table.mod-data tbody tr')
# print soup.find_all('td')

# print soup.prettify() 


# for i in soup.find_all(name="span"): 
#     print i



In [2]:
url = 'http://caselaw.findlaw.com/summary/search/?query=filters&court=us-1st-circuit&dateFormat=yyyyMMdd&topic=cs_42&pgnum=1'

content = requests.get(url).content
soup = BeautifulSoup(content, "html.parser")


def get_page_urls(soup): 
    """
    Get all the search result, case urls from a single page.
    :param soup: Beautiful soup object
    :return: url_list, the desired urls to be checked
    """
    url_list = []
    for link in soup.findAll('a', attrs={'href': re.compile("^http://")}):
        if 'circuit' in link.get('href'): 
            url_list.append(link.get('href'))
    return url_list

# url_list = []
# for link in soup.findAll('a', attrs={'href': re.compile("^http://")}):
#     if 'circuit' in link.get('href'): 
#         url_list.append(link.get('href'))

# for i in (url_list): 
#     print i





### Check For Working Urls Functions


In [3]:
### Function for checkin every site 

# url = 'http://caselaw.findlaw.com/summary/search/?query=filters&court=us-1st-circuit&dateFormat=yyyyMMdd&topic=cs_42&pgnum=1'

# content = requests.get(url).content
# soup = BeautifulSoup(content, "html.parser")

url_list = []
for link in soup.findAll('a', attrs={'href': re.compile("^http://")}):
    if 'circuit' in link.get('href'): 
        url_list.append(link.get('href'))
        
def get_next_url(current_url, soup):
    """
    Get the URL of the next listings page.
    :param soup: Beautiful soup object
    :return: string, the URL of the next webpage
    """
    
    d = [ i.get('href') for i in soup.find_all(name="a", attrs={"class": "pgnum"}) ][-1]
    return urljoin(current_url, d)

        
        
def create_soup(url):
    """
    Get the HTML contents of the URL.
    :param url: string, the url to scrape
    :return: soup: a BeautifulSoup object
    """
    page = requests.get(url)
    return BeautifulSoup(page.text, "html.parser")


def second_check(url):
    
    content = requests.get(url).content
    soup = BeautifulSoup(content, "html.parser")
    
    for i in soup.find_all("p"): 
#         print i, type(i)
        try: 
            href= i.a.get('href')
            return  href
            break
    
        except:
            continue 
            
def first_check(url_list): 
    ## Get case text from second layer link if in initial check, otherwise check a second time
    ## for an additional internal link in second check.
    ## Return confirmed links with case texts: 

    target_urls = []
    
    for site in url_list:
        new_soup = create_soup(site)

        for html in new_soup.find_all(name="div", attrs={"class": "btn_read"}): 
            check_url= html.a.get('href')        
            check_soup = create_soup(check_url)
            
            if check_soup.find_all("a", { "target" : "new"}): 
#                 print "NEEDED A SECOND CHECK"
                target_urls.append(second_check(check_url))
#                 print "BUT ADDED: ", second_check(check_url), "On the Second Check"

            else: 
                target_urls.append(check_url)
#                 print 'ON THE FIRST TRY, ADDED: ', check_url
        sleep(2)
    return target_urls

# first_check(url_list)



### Second Check Function


In [ ]:
url = 'http://caselaw.findlaw.com/summary/search/?query=filters&court=us-1st-circuit&dateFormat=yyyyMMdd&topic=cs_42&pgnum=1'
content = requests.get(url).content
soup = BeautifulSoup(content, "html.parser")

for i in soup.find_all(name="a", attrs={"class": "pgnum"}) 


print   

In [45]:
def run_scraper(current_url, dft):
    """
    Run the web scraper that will scrape Findlaw
    :param current_url: string, the initial URL to scrape
    :param dft: Pandas dataframe, the listings data from previous scrapings
    :return: dft: Pandas dataframe containing scraped information
    """
#     content = requests.get(url).content
#     soup = BeautifulSoup(content, "html.parser")
    

#     for i in soup.find_all(name="a", attrs={"class": "pgnum"}): 
#         next_list.append(i.text)
    
    flag = u'\xbb'
    list_of_urls = []
    soup = create_soup(current_url)

    # Run the scraper until it runs out of pages to scrape
    run = True
    
    while u'\xbb' in flag and run==True:
        ## "Keep Going; there was an error trying to make >> a string"
        ## (1) From soup fill next_list with page links at the bottom + next symbol 
        next_list =[]

        for i in soup.find_all(name="a", attrs={"class": "pgnum"}): 
            next_list.append(i.text)

#             flag = next_list[-1]

        ## (2) Get URLS using Check Functions 
        initial_url_list= get_page_urls(soup)
        confirmed_urls = first_check(initial_url_list)

        with open('output.txt', 'a') as f: 
            for link in confirmed_urls: 
                f.write(link + '\n')

#             dft.to_csv('confirmed_urls.txt', sep=' ', index=False, header=False)


        ## (3) Get court texts 


        ## (4) Get next page's pagination info 

        current_url = get_next_url(current_url, soup)         
        soup = create_soup(current_url)

        next_list =[]

        for i in soup.find_all(name="a", attrs={"class": "pgnum"}): 
            next_list.append(i.text)
            sleep(1)

        flag = next_list[-1]
        run = False
            
#         dft = dft.append(pd.DataFrame(listings), ignore_index=True)
        
    return 


# df_new = pd.DataFrame(columns=["job_title", "location", "company",
#                                    "url", "jobsite", "job_description"])

df = pd.DataFrame(columns=["url"])
first_url = 'http://caselaw.findlaw.com/summary/search/?query=filters&court=us-1st-circuit&dateFormat=yyyyMMdd&topic=cs_42&pgnum=1'
run_scraper(first_url, df)        


In [47]:
url = 'http://caselaw.findlaw.com/summary/search/?query=filters&court=us-1st-circuit&dateFormat=yyyyMMdd&topic=cs_42&pgnum=1'

content = requests.get(url).content
soup = BeautifulSoup(content, "html.parser")


# flag = "\xbb"
# list_of_urls = []

#     # Run the scraper until it runs out of pages to scrape

# while "\xbb" in flag:
#     soup = create_soup(url)
    
#     next_list =[]

#     for i in soup.find_all(name="a", attrs={"class": "pgnum"}): 
#         next_list.append(i.text)
#         print 

#     flag = next_list[-1]


next_list =[]

# str = unicode(str, errors='replace')


for i in soup.find_all(name="a", attrs={"class": "pgnum"}): 
    next_list.append(i.text)#.encode("utf-8")))


flag= next_list[-1]
    
# print next_list
# print flag , type(flag)

# flag = u'4'

#     try: 
#         flag = str(flag)
#         print flag, type(flag)
#         print 'You can BREAK from the while loop now; converting flag to a STRING was successful\
#      meaning that there are NO MORE next signs'

#     except: 

#         print "Keep Going; there was an error trying to make >> a string"

if u"\xbb" in flag: 
    print 'YESSSSS'


YESSSSS


In [16]:
# if "\xbb" in flag: 
#     print 'YESSSSS'

# # next_list= next_list[0:-1]



# try: 
#     type(int(next_list[-1])) != type(int)
    
#     print "Yes!!"
    
# except: 
#     print "No!!"
    

# print next_list

# while type(int(next_list[-1])) != type(int): 
#     for i in (range(1,5)):
#         next_list.append(i)
    
#     print next_list
#     next_list.append('i')
    
    
        

# print soup.find_all(name="a", attrs={"class": "pgnum"})




# print soup.find(name="span", attrs={"class": "pgnum"}).text
                
# for next_ in soup.find_all(name="span", attrs={"a": "title"}): 
#     print next_['title']

In [24]:
for next_ in soup.find_all(name="span", attrs={"a": "title"}): 
    print next_['title']

In [87]:
len(next_list)

6

In [ ]:
url = 'http://caselaw.findlaw.com/summary/search/?query=filters&court=us-1st-circuit&dateFormat=yyyyMMdd&topic=cs_42&pgnum=1'

content = requests.get(url).content
soup = BeautifulSoup(content, "html.parser")

next_list =[]
  
while type(int(next_list[-1])) != type(int): 
    for i in (range(1,5)):
        next_list.append(i)
    
    next_list.append('i')
    
print next_list
    

In [5]:
## Testing Collections/ Dictionary
from collections import defaultdict 

In [14]:
my_dict = defaultdict(list)

ex= [["DS", "dallas", "...."], ['data scientist', 'austin', 'blah...'] ]
for i in ex: 
    for ind, e in enumerate(i): 
#         print e
        if ind==0: 
            my_dict['Job Title']+= [e]
        elif ind==1: 
            my_dict['City']+= [e]
        else: 
            my_dict['Job Info']+= [e]
        
print my_dict



defaultdict(<type 'list'>, {'City': ['dallas', 'austin'], 'Job Info': ['....', 'blah...'], 'Job Title': ['DS', 'data scientist']})
